## Problema do Projeto de Viga Soldada _(Welded Beam Design)_
### Um problema de Engenharia Estrutural

A engenharia estrutural, em sua essência, é a busca pelo equilíbrio entre segurança, funcionalidade e economia. O problema que estamos tratando foi formulado seminalmente por __K. M. Ragsdell e D. T. Phillips__ em 1976, através do artigo _"Optimal Design of a Class of Welded Structures Using Geometric Programming"_, publicado no _"Journal of Engineering for Industry"_. Eles propuseram a aplicação da Programação Geométrica _(Geometric Programming)_ para resolver problemas de design com restrições não-lineares complexas. O método busca minimizar o custo e modelar falhas físicas, transformando estes fenômenos em inequações algébricas tratáveis. Em seguida serão analiasadas cada variável, cada termo da função objetivo e a física subjacente a cada restrição. Mais detalhes podem ser obtidos na nossa apresentação, clicando aqui.

### Descrição do problema:

 Trata-se de uma viga rígida de aço, de seção transversal retangular, que deve ser fixada a um suporte rígido, como uma coluna ou parede; a fixação é realizada através de soldagem. A viga é projetada para suportar uma carga pontual $P$ aplicada em sua extremidade livre. A eficácia do conjunto depende de suas dimensões e da qualidade da solda. __O problema de otimização reside na determinação das dimensões exatas que minimizem o custo de fabricação sem que a estrutura falhe__. Devido à natureza altamente não-linear das restrições do problema e à complexidade da superfície de busca, o problema da viga soldada tornou-se um "padrão-ouro" (benchmark) para testar a eficácia de novos algoritmos de otimização.

### Variáveis (Vetor de Projeto)

Este problema é definido por quatro variáveis contínuas. Para este relatório, adotaremos a notação vetorial $${x} = [x_1, x_2, x_3, x_4]^T$$ 
| Variável | Descrição | 
|:---------|:------:|
| $x_1$ |   __Espessura da solda:__ Dimensão das pernas do filete de solda. É a dimensão crítica que define a área resistente ao cisalhamento     |
| $x_2$       |   __Comprimento da solda:__ O comprimento onde a viga toca o suporte, onde a solda é aplicada. Afeta tanto a resistência da solda quanto o braço de alavanca   | 
| $x_3$       |    __Altura da viga:__ Dimensão vertical da viga. Variável mais influente na resistência à deflexão    |
| $x_4$       |   __Largura da viga:__ A dimensão horizontal da viga. Fundamental para a estabilidade lateral (flambagem)    |    


### Definição do Espaço de Busca

Usaremos os limites padrão utilizados na maioria dos estudos contemporâneos (e.g., bibliotecas como _NEORL_ ou _MATLAB GADS_): $x_1, x_4 \in [0.1, 2.0]$ e $x_2, x_3 \in [0.1, 10.0]$

| Variável | Limite Inferior| Limite Superior |
|:---------|:------|:------:|
| $x_1$ |   0.1    |   2.0    |
| $x_2$ |   0.1    |   10.0   | 
| $x_3$ |   0.1    |   10.0   | 
| $x_4$ |   0.1    |   2.0    |

### Função objetivo

O objetivo central do problema é econômico: minimizar o custo total de fabricação. Ragsdell e Phillips modelaram este custo como a soma de dois componentes principais: o custo do material da viga e o custo da mão-de-obra e material da soldagem. A função objetivo $f({x})$ é expressa como: $f({x}) = C_{solda} + C_{viga}$

Expandindo os termos, temos $f({x}) = (c_1 + c_0) \cdot V_{solda} + c_2 \cdot V_{viga}$, onde:

$V_{solda} \approx x_1^2 \cdot x_2$ 

$V_{viga} = x_3 \cdot x_4 \cdot (L + x_2)$ 

$c_0$: Custo de instalação 

$c_1$: Custo pela solda (material + mão-de-obra)

$c_2$: Custo pela barra de aço

Expandindo novamente, temos:
$ (c_1 + c_0) \cdot (x_1^2 \cdot x_2) + c_2 \cdot (x_3 \cdot x_4 \cdot (L + x_2)) $

Na formulação padrão, os custos unitários são coeficientes numéricos fixos, resultando na __equação canônica:__ $$f({x}) = 1.10471 x_1^2 x_2 + 0.04811 x_3 x_4 (14.0 + x_2)$$

__Nota:__ estes coeficientes numéricos foram encontrados através de uma pesquisa de mercado realizada por Ragsdell e Phillips na década de 70. 

#### Trade-offs: 
- __O Alto Custo da Soldagem ($1.10471$):__ Reflete a realidade industrial onde a mão-de-obra especializada e os consumíveis de soldagem são significativamente caros.
__Implicação: O otimizador será pressionado a reduzir $x_1$ (espessura) e $x_2$ (comprimento) para baixar o custo relacionado à solda, trazendo um risco: comprometer a resistência ao cisalhamento (restrição $g_1$)__
- __A Dependência Quadrática ($x_1^2$):__ O custo da solda cresce quadraticamente com a espessura ($x_1$). Dobrar a espessura da solda quadruplica seu volume e custo.
__Implicação: É economicamente preferível ter uma solda mais longa ($x_2$) e fina ($x_1$) do que uma solda curta e grossa, desde que a geometria permita__
- __O Custo do Material ($x_3, x_4$):__ O custo da viga é linear em relação às suas dimensões.
__Implicação: Aumentar a altura da viga ($x_3$) aumenta o custo linearmente, mas aumenta a inércia (resistência à flexão) cubicamente, o que favorece vigas altas e finas__

### Modelagem das restrições:

As restrições $g_j({x}) \leq 0$ representam modos de falha física. A violação de uma restrição implica o colapso da estrutura. Existem sete restrições principais no modelo padrão. O completo entendimento das restrições implica conhecimento profundo em engenharia e física; para este projeto, vamos considerá-las como "verdades universais"

#### Constantes Físicas:
- Carga Aplicada ($P$): 6.000 libras-força
- Comprimento do Balanço ($L$): 14 polegadas
- Módulo de Elasticidade ($E$): $30 \times 10^6$ Libras por polegada quadrada (psi)
- Módulo de Cisalhamento ($G$): $12 \times 10^6$ psi 
- Tensão Máxima de Cisalhamento ($\tau_{max}$): 13.600 psi
- Tensão Máxima de Flexão ($\sigma_{max}$): 30.000 psi
- Deflexão Máxima Permitida ($\delta_{max}$): 0.25 polegadas

#### Restrições:

- __Restrição $g_1$: Tensão de Cisalhamento na Solda__

__Tensão de Cisalhamento Primária ($\tau'$):__  $$\tau' = \frac{P}{\sqrt{2} x_1 x_2}$$
__Tensão de Cisalhamento Secundária ($\tau''$):__ $$\tau'' = \frac{M \cdot R}{J}$$
Momento ($M$): $$M = P \cdot (L + \frac{x_2}{2})$$
Raio Polar ($R$): $$R = \sqrt{ \left(\frac{x_2}{2}\right)^2 + \left(\frac{x_1 + x_3}{2}\right)^2 }$$
Momento Polar de Inércia ($J$): $$J = 2 \left\{ \sqrt{2} x_1 x_2 \left[ \frac{x_2^2}{12} + \left(\frac{x_1 + x_3}{2}\right)^2 \right] \right\}$$
__Tensão Combinada ($\tau$):__ $$\tau({x}) = \sqrt{ (\tau')^2 + (\tau'')^2 + 2 \tau' \tau'' \frac{x_2}{2R} }$$

__A Inequação ($g_1$):__ $\tau({x}) - \tau_{max} \leq 0$

- __Restrição $g_2$: Tensão de Flexão na Viga__
Momento Fletor Máximo: $$M_{viga} = P \cdot L = 6000 \cdot 14$$
Módulo de Resistência ($S$): $$S = \frac{x_4 x_3^2}{6}$$
Tensão Normal ($\sigma$): $$\sigma({x}) = \frac{6 P L}{x_4 x_3^2}   ou   \frac{504.000}{x_4 x_3^2}$$

__A Inequação ($g_2$):__ $g_2({x}) = \sigma({x}) - \sigma_{max} \leq 0$

- __Restrição $g_3$ - Compatibilidade Geométrica Solda-Viga:__
Uma restrição prática de fabricação: a espessura do cordão de solda ($x_1$) não pode ser maior que a largura da viga ($x_4$) onde ela é aplicada.

__A Inequação ($g_3$):__ $g_3({x}) = x_1 - x_4 \leq 0$

- __Restrição $g_4$ - Restrição de Custo:__
Esta restrição força o algoritmo a encontrar soluções abaixo de um valor arbitrário, que representa unidades monetárias, criando uma região viável artificialmente limitada pelo orçamento. Frequentemente este valor é associado a 5.0

__A Inequação ($g_4$):__ $g_4({x}) = 1.10471 x_1^2 x_2 + 0.04811 x_3 x_4 (14.0 + x_2) - 5.0 \leq 0$

- __Restrição $g_5$ - Tamanho Mínimo da Solda:__
Para garantir a integridade da solda, impõe-se um limite inferior. Embora o espaço de busca já defina $x_1 \geq 0.1$, esta restrição é frequentemente adicionada para forçar um valor mínimo viável, como 0.125 (1/8 polegada)

__A Inequação ($g_5$):__ $g_5({x}) = 0.125 - x_1 \leq 0$

- __Restrição $g_6$ - Deflexão da Ponta:__
Para garantir a utilidade da estrutura, a viga não deve se deformar excessivamente. Caso ocorra, pode inutilizar o equipamento $$\delta({x}) = \frac{4 P L^3}{E x_4 x_3^3} ou \frac{65.856.000}{30 \times 10^6 x_4 x_3^3}$$

__A Inequação ($g_6$):__ $g_6({x}) = \delta({x}) - \delta_{max} \leq 0$

- __Restrição $g_7$ - Flambagem Lateral-Torsional:__
Ragsdell e Phillips utilizaram uma aproximação para a carga crítica de flambagem ($P_c$) derivada da teoria de estabilidade elástica de Timoshenko $$P_c({x}) = \frac{4.013 E \cdot x_3 x_4^3}{6 L^2} \left( 1 - \frac{x_3}{2L} \sqrt{\frac{E}{4G}} \right)$$
Numericamente, inserindo as constantes: $$P_c({x}) = 64746.022 \cdot (1 - 0.0282346 \cdot x_3) \cdot x_3 \cdot x_4^3$$

__A Inequação ($g_7$):__ $6000 - P_c({x}) \leq 0$

Esta restrição é o "antídoto" para a restrição de flexão ($g_2$). Enquanto $g_2$ pede uma viga alta e fina, $g_7$ pune severamente vigas finas. O ótimo global do problema reside, portanto, no limite preciso onde a viga é alta o suficiente para não falhar por deflexão, mas larga o suficiente para não flambar.


## Implementação do código
#### Chegamos na parte prática do projeto, onde vamos implementar dois métodos diferentes para resolver o problema formulado e descrito acima. Para fins de comparação e correção, buscamos o valor de consenso para o ótimo global "verdadeiro": na época, o melhor global era de __2.3809 a 2.3810__; atualmente, o melhor global já está em __~1.72__. Estes valores podem ser encontrados em vários _papers_ relacionados ao tema.


### Definição do problema e fórmulas físicas
Antes de implementar os algoritmos de otimização, vamos definir em código o nosso problema e as fórmulas físicas necessárias para resolvê-lo.

__Nota: algumas restrições estão normalizadas devido ao método de minimização do Lagrangiano Aumentado.__ Como a ordem das restrições varia bastante (de $10¹$ a $10⁴$) e o método é sensível à isso, ele não muda algumas variáveis e acaba por nunca convergir.

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

plt.style.use('seaborn-v0_8-whitegrid')

In [144]:
class WeldedBeamProblem:
    def __init__(self):
        # Constantes Físicas
        self.P = 6000.0        # Carga 
        self.L = 14.0          # Comprimento 
        self.E = 30e6          # Módulo de elasticidade
        self.G = 12e6          # Módulo de cisalhamento
        self.tau_max = 13600   # Tensão max cisalhamento 
        self.sigma_max = 30000 # Tensão max flexão 
        self.delta_max = 0.25  # Deflexão max 

        # Espaço de busca
        self.bounds = [
            (0.1, 2.0),   # x1: Espessura da solda
            (0.1, 10.0),  # x2: Comprimento da solda
            (0.1, 10.0),  # x3: Altura da viga
            (0.1, 2.0)    # x4: Largura da viga
        ]

    # Recebe um vetor x com os valores das variáveis
    def objective_function(self, x):
        x1, x2, x3, x4 = x
        return 1.10471 * (x1**2) * x2 + 0.04811 * x3 * x4 * (14.0 + x2)

    def calc_physics(self, x):
        """
        Calcula e retorna um dicionário com todas as grandezas físicas reais.
        Útil para evitar repetição de fórmulas.
        """
        x1, x2, x3, x4 = x
        P, L, E, G = self.P, self.L, self.E, self.G

        # Cisalhamento
        tau_prime = P / (np.sqrt(2) * x1 * x2)
        M = P * (L + (x2 / 2))
        R = np.sqrt((x2**2 / 4) + ((x1 + x3) / 2)**2)
        J = 2 * (np.sqrt(2) * x1 * x2 * ( (x2**2 / 12) + ((x1 + x3) / 2)**2 ))
        tau_double_prime = (M * R) / J
        tau_total = np.sqrt(tau_prime**2 + tau_double_prime**2 + 2 * tau_prime * tau_double_prime * (x2 / (2*R)))

        # Flexão
        sigma = (6 * P * L) / (x4 * x3**2)

        # Deflexão
        delta = (4 * P * L**3) / (E * x4 * x3**3)

        # Flambagem
        Pc = 64746.022 * (1 - 0.0282346 * x3) * x3 * x4**3
        
        return {
            "tau": tau_total,
            "sigma": sigma,
            "delta": delta,
            "Pc": Pc,
            "cost": self.objective_function(x)
        }

    def constraints(self, x):
        """
        Retorna as restrições normalizadas para o otimizador.
        """
        # 1. Busca os valores físicos já calculados
        phys = self.calc_physics(x)
        x1, x2, x3, x4 = x
        
        # 2. Normaliza para o formato g(x) <= 0
        g1 = (phys["tau"] / self.tau_max) - 1.0
        g2 = (phys["sigma"] / self.sigma_max) - 1.0
        g3 = x1 - x4
        g4 = (phys["cost"] / 5.0) - 1.0
        g5 = 0.125 - x1
        g6 = (phys["delta"] / self.delta_max) - 1.0
        # Evita divisão por zero na flambagem se Pc for muito pequeno
        g7 = (self.P / phys["Pc"]) - 1.0 if phys["Pc"] > 1e-5 else 100.0 

        return np.array([g1, g2, g3, g4, g5, g6, g7])

### Método Lagrangiano Aumentado (Visto em aula)
Optamos por este método como primeira escolha pois as outras opções vistas em aula são mais eficazes em problemas irrestritos. O método escolhido transforma o problema restrito em uma sequência de problemas irrestritos suaves.

#### Implementação do algoritmo

In [151]:
def augmented_lagrangian_method(problem, tol, max_iter=100):
    # Inicializa as variáveis com valores 
    # aleatórios dentro do espaço de busca
    bounds = problem.bounds
    x0 = [np.random.uniform(b[0], b[1]) for b in bounds]
    
    # Inicializa os parâmetros
    n_constr = 7 
    mu = np.zeros(n_constr) 
    rho = 1.0
    mu_max = 1e20 
    
    # Transforma a lista em um vetor operável
    x_k = np.array(x0)

    # Controle da atualização do Rho
    tau = 0.5     
    gamma = 10.0 
    prev_violation_metric = float('inf') 
    
    # Formatação em tabela
    print(f"{'Iter':<5} | {'Custo f(x)':<12} | {'Max Violação':<12} | {'Rho':<10}")
    print("-" * 50)

    # Variável para guardar o número final de iterações
    final_iter_count = 0

    for k in range(max_iter):
        final_iter_count = k + 1 
        
        # definição da função
        def phr_function(x):
            f = problem.objective_function(x)
            g = problem.constraints(x) 
            
            # Cálculo da penalidade
            penalty_sum = 0
            for i in range(n_constr):
                val = max(0, g[i] + (mu[i] / rho))**2 
                penalty_sum += val
                
            return f + (rho / 2.0) * penalty_sum

        # Minimização: como não foi especificado um método específico,
        # optamos por usar o BFGS, que se encaixa bem no problema
        res = minimize(phr_function, 
                       x_k, 
                       method='L-BFGS-B', 
                       bounds=bounds,
                       options={'ftol': 1e-6})
        
        x_next = res.x
        
        # Avaliação do novo ponto
        g_val = problem.constraints(x_next)
        violation = np.max(g_val)
        f_val = problem.objective_function(x_next)
        
        # Logs para verficar o andamento
        print(f"{k:<5} | {f_val:<12.4f} | {violation:<12.6f} | {rho:<10.1f}")
        
        # Critério de Parada 
        if violation <= tol and np.linalg.norm(x_next - x_k) < tol:
            print("\nFunção convergiu com sucesso!")
            return x_next, f_val, final_iter_count
        
        # Atualização do Rho
        V_vector = np.maximum(g_val, -mu / rho)
        current_violation_metric = np.max(np.abs(V_vector))

        # Regra de Atualização do Rho 
        next_rho = rho 
        if k == 0:
            pass 
        else:
            if current_violation_metric <= tau * prev_violation_metric:
                pass 
            else:
                next_rho = rho * gamma 
        
        # Atualização dos multiplicadores
        for i in range(n_constr):
            mu[i] = min(max(0, mu[i] + rho * g_val[i]), mu_max)
        
        # Aplica o novo rho e atualiza métricas para a próxima rodada
        rho = next_rho
        prev_violation_metric = current_violation_metric
        x_k = x_next

    return x_k, problem.objective_function(x_k), max_iter

#### Execução do algoritmo


In [152]:
# Instancia o problema
problema = WeldedBeamProblem()

# Roda o algoritmo
tol = 1e-6
x_opt, f_opt, n_iters = augmented_lagrangian_method(problema, tol)

print(f"O método convergiu em {n_iters} iterações.")
print(f"Custo Final: {f_opt:.4f}")

Iter  | Custo f(x)   | Max Violação | Rho       
--------------------------------------------------
0     | 1.5164       | 0.306324     | 1.0       
1     | 1.7029       | 0.115549     | 1.0       
2     | 1.7724       | 0.064495     | 1.0       
3     | 1.8330       | 0.031379     | 10.0      
4     | 1.8350       | 0.031131     | 10.0      
5     | 1.8611       | 0.000807     | 100.0     
6     | 1.8610       | 0.000676     | 100.0     
7     | 1.8610       | 0.000666     | 1000.0    
8     | 1.8622       | 0.000502     | 10000.0   
9     | 1.8625       | 0.000006     | 100000.0  
10    | 1.8623       | 0.000030     | 100000.0  
11    | 1.8623       | -0.000001    | 100000.0  
12    | 1.8623       | -0.000000    | 1000000.0 

Função convergiu com sucesso!
O método convergiu em 13 iterações.
Custo Final: 1.8623


#### Verificação das restrições físicas

In [153]:
print(f"Custo Encontrado: {f_opt:.5f}")
print("-" * 30)

props = problema.calc_physics(x_opt)

print(f"Variáveis Otimizadas:")
print(f"  x1 (Solda h): {x_opt[0]:.4f} in")
print(f"  x2 (Solda l): {x_opt[1]:.4f} in")
print(f"  x3 (Viga t):  {x_opt[2]:.4f} in")
print(f"  x4 (Viga b):  {x_opt[3]:.4f} in")
print("-" * 30)

print("Verificação das Restrições Físicas:")

# Tolerâncias
tol_tensao = 1.0     
tol_geom = 1e-3      
tol_carga = 1.0      

# Valores
tau   = props["tau"]
sigma = props["sigma"]
delta = props["delta"]
Pc    = props["Pc"]
x1, x4 = x_opt[0], x_opt[3]

print(f"1. Cisalhamento (tau): {tau:.1f} <= 13600 psi? "
      f"{'[OK]' if tau <= 13600 + tol_tensao else '[FALHA]'}")

print(f"2. Flexão (sigma):     {sigma:.1f} <= 30000 psi? "
      f"{'[OK]' if sigma <= 30000 + tol_tensao else '[FALHA]'}")

print(f"3. Geometria (h <= b): {x1:.4f} <= {x4:.4f}?      "
      f"{'[OK]' if x1 <= x4 + tol_geom else '[FALHA]'}")

print(f"6. Deflexão (delta):   {delta:.4f} <= 0.25 in?     "
      f"{'[OK]' if delta <= 0.25 + tol_geom else '[FALHA]'}")

print(f"7. Flambagem (Pc):     {Pc:.1f} >= 6000 lb?    "
      f"{'[OK]' if Pc >= 6000 - tol_carga else '[FALHA]'}")

Custo Encontrado: 1.86232
------------------------------
Variáveis Otimizadas:
  x1 (Solda h): 0.2441 in
  x2 (Solda l): 3.0435 in
  x3 (Viga t):  8.2955 in
  x4 (Viga b):  0.2443 in
------------------------------
Verificação das Restrições Físicas:
1. Cisalhamento (tau): 13600.0 <= 13600 psi? [OK]
2. Flexão (sigma):     29974.0 <= 30000 psi? [OK]
3. Geometria (h <= b): 0.2441 <= 0.2443?      [OK]
6. Deflexão (delta):   0.0157 <= 0.25 in?     [OK]
7. Flambagem (Pc):     6000.2 >= 6000 lb?    [OK]


### Método Sequential Quadratic Programming (SQP)

Escolhemos este método como segunda implementação devido à sua eficiência em lidar com funções objetivo e restrições suaves, restrições de desigualdade fortemente não-lineares e problemas onde a precisão numérica é crítica.

#### Implementação do algoritmo SQP (via biblioteca scipy)

A biblioteca já possui este método implementado. Assim, focamos em explicar o método e garantir sua convergência. A explicação está abaixo do código.

In [201]:
def solve_with_sqp(problem):
    # Ponto Inicial Aleatório 
    x0 = [np.random.uniform(b[0], b[1]) for b in problem.bounds]
    
    # Adapta as restrições para seguirem o formato do scipy
    def constraint_adapter(x):
        return -1.0 * problem.constraints(x)

    # Configuração do Scipy
    cons = {'type': 'ineq', 'fun': constraint_adapter}

    # Execução do Solver SLSQP
    res = minimize(
        problem.objective_function,
        x0,
        method='SLSQP',           
        bounds=problem.bounds,    
        constraints=cons,         
        options={'ftol': 1e-6, 'disp': False, 'maxiter': 100}
    )

    # Resultados
    x_opt = res.x
    f_opt = res.fun
    n_iters = res.nit 
    
    return x_opt, f_opt, n_iters

#### Execução do algoritmo

In [230]:
# Chamada da função
x_sqp, f_sqp, iter_sqp = solve_with_sqp(problema)
print(f"Iterações até convergir: {iter_sqp}")
print(f"Custo Mínimo encontrado: {f_sqp:.5f}")

# Recuperar física 
props_sqp = problema.calc_physics(x_sqp)
tau   = props_sqp["tau"]
sigma = props_sqp["sigma"]
delta = props_sqp["delta"]
Pc    = props_sqp["Pc"]
x1, x4 = x_sqp[0], x_sqp[3]

# Tolerâncias
tol_tensao = 1.0     
tol_geom = 1e-3      
tol_carga = 1.0      

print(f"1. Cisalhamento (tau): {tau:.1f} <= 13600 psi? "
      f"{'[OK]' if tau <= 13600 + tol_tensao else '[FALHA]'}")

print(f"2. Flexão (sigma):     {sigma:.1f} <= 30000 psi? "
      f"{'[OK]' if sigma <= 30000 + tol_tensao else '[FALHA]'}")

print(f"3. Geometria (h <= b): {x1:.4f} <= {x4:.4f}?      "
      f"{'[OK]' if x1 <= x4 + tol_geom else '[FALHA]'}")

print(f"6. Deflexão (delta):   {delta:.4f} <= 0.25 in?     "
      f"{'[OK]' if delta <= 0.25 + tol_geom else '[FALHA]'}")

print(f"7. Flambagem (Pc):     {Pc:.1f} >= 6000 lb?    "
      f"{'[OK]' if Pc >= 6000 - tol_carga else '[FALHA]'}")

Iterações até convergir: 10
Custo Mínimo encontrado: 1.86164
1. Cisalhamento (tau): 13600.0 <= 13600 psi? [OK]
2. Flexão (sigma):     30000.0 <= 30000 psi? [OK]
3. Geometria (h <= b): 0.2444 <= 0.2444?      [OK]
6. Deflexão (delta):   0.0158 <= 0.25 in?     [OK]
7. Flambagem (Pc):     6000.0 >= 6000 lb?    [OK]


#### Funcionamento:

__A ideia central__ do SQP é aplicar o Método de Newton diretamente às condições de otimalidade (KKT) do problema restrito. Em vez de resolver o problema difícil, o algoritmo resolve uma sequência de subproblemas quadráticos mais simples.

A cada iteração $k$, o algoritmo realiza:
- Aproximação Quadrática: Constrói-se uma aproximação quadrática da Função Lagrangiana e uma linearização das restrições em torno do ponto atual $x_k$​

- Subproblema QP: Resolve-se um subproblema de Programação Quadrática para encontrar uma direção de busca $d_k$​
$$ \text{Minimizar: } \frac{1}{2} d_k^T H_k d_k + \nabla f(x_k)^T d_k $$
$$ \text{sujeito a: } \nabla g(x_k)^T d_k + g(x_k) \le 0 $$

- Atualização da Hessiana: A matriz $H_k$​ é atualizada iterativamente usando métodos Quasi-Newton (geralmente BFGS), evitando calcular derivadas segundas exatas

- Função de Mérito e Busca Linear: Para garantir que o passo $d_k$​ realmente leve a uma solução melhor e viável, o algoritmo utiliza uma "Função de Mérito" (L1) para decidir o tamanho do passo $\alpha_k$​

Atualização: $x_{k+1} = x_k + \alpha_k d_k$

__Garantias de Convergência__

O SQP possui propriedades teóricas robustas que o tornam superior a métodos de penalidade simples em muitos casos:
- Convergência Local: Se o ponto inicial $x_0$​ estiver suficientemente próximo da solução ótima $x^∗$, e assumindo que as condições de Qualificação de Restrições (LICQ) e Suficiência de Segunda Ordem sejam satisfeitas, o método converge com uma taxa superlinear;

- Convergência Global: Quando implementado com uma estratégia de globalização (como a função de mérito que usaremos), o teorema garante que o algoritmo convergirá para um ponto estacionário (ponto KKT) a partir de qualquer ponto inicial arbitrário, mesmo que distante da solução.

## Artigos e papers usados para encontrar, modelar, desenvolver e resolver este problema:

https://doi.org/10.1115/1.3438995

https://doi.org/10.2514/3.10834

https://doi.org/10.1016/S0166-3615(99)00046-9

https://doi.org/10.1016/j.cma.2004.09.007

https://doi.org/10.1109/TEVC.2003.814902